In [4]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import euclidean
from scipy.spatial import distance_matrix
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler 

In [5]:

data = pd.read_excel("Taux_Remplissage_ComCom_Nebbiu.xlsx")
data = data.drop(['Unnamed: 0'], axis=1)
date = data['Unnamed: 1']
#data = data.drop(['Unnamed: 1'], axis=1)
merged_data = pd.DataFrame()

print(data.head())
gps = [data[i][1] for i in data.columns if i.startswith('poubelle')]


def dms_to_decimal(degrees, minutes, seconds, direction):
    decimal_degrees = float(degrees) + float(minutes) / \
        60 + float(seconds)/(60*60)
    if direction in ['S', 'W']:
        decimal_degrees = -decimal_degrees
    return decimal_degrees


def parse_coordinates(coord_string:str)->tuple:
    '''convertit les coordonnées GPS en degrés décimaux'''
    degrees = re.findall(r"(\d+)°", coord_string)
    minutes = re.findall(r'(\d+)\'', coord_string)
    seconds = re.findall(r'(\d+)"', coord_string)
    lat_dir = re.findall(r'([NSEW])+', coord_string)
    
    latitude = dms_to_decimal(degrees[0], minutes[0], seconds[0], lat_dir[0])
    longitude = dms_to_decimal(degrees[1], minutes[1], seconds[1], lat_dir[1])

    return latitude, longitude

coordinates = [parse_coordinates(coord) for coord in gps]
print(coordinates)

            Unnamed: 1            poubelle 1   Unnamed: 3      Unnamed: 4  \
0                 date                   GPS  remplissage  coeff touriste   
1  2023-01-01 00:00:00  42°36'39"N 9°21'14"E            5               1   
2  2023-01-02 00:00:00                   NaN           40               1   
3  2023-01-03 00:00:00                   NaN            0               1   
4  2023-01-04 00:00:00                   NaN           37               1   

             poubelle 2    Unnamed: 6      Unnamed: 7            poubelle 3  \
0                  GPS   remplissage   coeff touriste                  GPS    
1  42°36'44"N 9°21'24"E            18               1  42°36'51"N 9°21'17"E   
2                   NaN            51               1                   NaN   
3                   NaN             0               1                   NaN   
4                   NaN            25               1                   NaN   

     Unnamed: 9     Unnamed: 10  ...     Unnamed: 13          

In [6]:
scaler = StandardScaler()
data[['remplissage', 'densite_population', 'affluence_touristique']] = scaler.fit_transform(data[['niveau_remplissage', 'densite_population', 'affluence_touristique']])

KeyError: "None of [Index(['niveau_remplissage', 'densite_population', 'affluence_touristique'], dtype='object')] are in the [columns]"

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

NameError: name 'X' is not defined

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

In [ ]:
new_data = pd.DataFrame({'niveau_remplissage': [0.8], 'densite_population': [1.2], 'affluence_touristique': [0.5]})
scaled_data = scaler.transform(new_data)
prediction = model.predict(scaled_data)
print(f'Prédiction de la fréquence de collecte : {prediction}')